## Preprocessing

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [237]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
app_df = application_df.drop(columns=["EIN","NAME", "SPECIAL_CONSIDERATIONS", "ORGANIZATION", "STATUS", "AFFILIATION"])
app_df

,APPLICATION_TYPE,CLASSIFICATION,USE_CASE,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,C1000,ProductDev,0,5000,1
1,T3,C2000,Preservation,1-9999,108590,1
2,T5,C3000,ProductDev,0,5000,0
3,T3,C2000,Preservation,10000-24999,6692,1
4,T3,C1000,Heathcare,100000-499999,142590,1
...,...,...,...,...,...,...
34294,T4,C1000,ProductDev,0,5000,0
34295,T4,C3000,ProductDev,0,5000,0
34296,T3,C2000,Preservation,0,5000,0
34297,T5,C3000,ProductDev,0,5000,1


In [238]:
# Determine the number of unique values in each column.
app_df.nunique()

APPLICATION_TYPE       9
CLASSIFICATION         8
USE_CASE               5
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

In [239]:
# Look at APPLICATION_TYPE value counts for binning
app_df.value_counts(["APPLICATION_TYPE"])

APPLICATION_TYPE
T3                  27037
T4                   1542
T6                   1216
T5                   1173
T19                  1065
T8                    737
T7                    725
T10                   528
Other                 276
dtype: int64

In [240]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ['T9', 'T13', 'T12', 'T2', 'T14', 'T25', 'T29', 'T15', 'T17']

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [241]:
# Look at CLASSIFICATION value counts for binning
app_df.value_counts(["CLASSIFICATION"])

CLASSIFICATION
C1000             17326
C2000              6074
C1200              4837
C3000              1918
C2100              1883
Other              1197
C7000               777
C1700               287
dtype: int64

In [242]:
# You may find it helpful to look at CLASSIFICATION value counts >1
app_df["CLASSIFICATION"].value_counts().loc[lambda x : x>150]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1197
C7000      777
C1700      287
Name: CLASSIFICATION, dtype: int64

In [243]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
filtered_values = app_df["CLASSIFICATION"].value_counts().loc[lambda x: x < 200]

classifications_to_replace = filtered_values.index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1197
C7000      777
C1700      287
Name: CLASSIFICATION, dtype: int64

In [244]:
# Convert categorical data to numeric with `pd.get_dummies`
categorical_columns = ['CLASSIFICATION', 'APPLICATION_TYPE']
dummy_variables = pd.get_dummies(app_df[categorical_columns])
print(dummy_variables)

       CLASSIFICATION_C1000  CLASSIFICATION_C1200  CLASSIFICATION_C1700  \
0                         1                     0                     0   
1                         0                     0                     0   
2                         0                     0                     0   
3                         0                     0                     0   
4                         1                     0                     0   
...                     ...                   ...                   ...   
34294                     1                     0                     0   
34295                     0                     0                     0   
34296                     0                     0                     0   
34297                     0                     0                     0   
34298                     1                     0                     0   

       CLASSIFICATION_C2000  CLASSIFICATION_C2100  CLASSIFICATION_C3000  \
0                       

In [245]:
# Split our preprocessed data into our features and target arrays
numeric_columns = dummy_variables.columns.difference(categorical_columns)
y = app_df["IS_SUCCESSFUL"]
X = dummy_variables[numeric_columns]

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [246]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [278]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=17, activation="relu", input_dim=17))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=7, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_57"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_167 (Dense)           (None, 17)                306       
                                                                 
 dense_168 (Dense)           (None, 7)                 126       
                                                                 
 dense_169 (Dense)           (None, 1)                 8         
                                                                 
Total params: 440
Trainable params: 440
Non-trainable params: 0
_________________________________________________________________


In [279]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [280]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=30)


Epoch 1/30
804/804 [==============================] - 1s 1ms/step - loss: 0.6524 - accuracy: 0.6023
Epoch 2/30
804/804 [==============================] - 1s 1ms/step - loss: 0.6422 - accuracy: 0.6138
Epoch 3/30
804/804 [==============================] - 1s 1ms/step - loss: 0.6418 - accuracy: 0.6144
Epoch 4/30
804/804 [==============================] - 1s 1ms/step - loss: 0.6419 - accuracy: 0.6137
Epoch 5/30
804/804 [==============================] - 1s 1ms/step - loss: 0.6415 - accuracy: 0.6134
Epoch 6/30
804/804 [==============================] - 1s 1ms/step - loss: 0.6416 - accuracy: 0.6144
Epoch 7/30
804/804 [==============================] - 1s 1ms/step - loss: 0.6415 - accuracy: 0.6136
Epoch 8/30
804/804 [==============================] - 1s 1ms/step - loss: 0.6413 - accuracy: 0.6150
Epoch 9/30
804/804 [==============================] - 1s 1ms/step - loss: 0.6416 - accuracy: 0.6155
Epoch 10/30
804/804 [==============================] - 1s 1ms/step - loss: 0.6413 - accuracy: 0.6131

In [282]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6429 - accuracy: 0.6155 - 308ms/epoch - 1ms/step
Loss: 0.6428900361061096, Accuracy: 0.6155102252960205


In [284]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity.h5')